In [1]:
import numpy as np
import pandas as pd

In [33]:
train = pd.read_csv('data/train.csv')

# test is the test set on which we make the final prediction
test = pd.read_csv('data/test.csv')

# Shape and preview
print('Train set shape:', train.shape)
print('Test set shape:', test.shape)
train.head()

# Repalace nans with dummies
train['Name']  = train.Name.fillna('Unknown Unknown')
train['Cabin'] = train.Cabin.fillna('ZZ/9999/ZZ')

train['Floor'] = train['Cabin'].map(lambda cabin: (cabin.split('/'))[0])
train['Seat']  = train['Cabin'].map(lambda cabin: (cabin.split('/'))[1])
train['Side']  = train['Cabin'].map(lambda cabin: (cabin.split('/'))[2])


train = train.drop(['Cabin'], axis=1)

# Split PassengerId into group and seat
train['Group'] = train['PassengerId'].map(lambda ID: ID.split('_')[0])
train['Num']   = train['PassengerId'].map(lambda ID: ID.split('_')[1])

train = train.drop(['PassengerId'], axis=1)

# group -> indications for name
train['Name'] = train['Name'].map(lambda name: name.split(' ')[1])


X = train.copy()

Train set shape: (8693, 14)
Test set shape: (4277, 13)


In [34]:
train['Group'].value_counts()
X.Group.value_counts()

4498    8
8168    8
8728    8
8796    8
8956    8
       ..
3483    1
3480    1
3478    1
3473    1
4620    1
Name: Group, Length: 6217, dtype: int64

In [14]:
# Transform age:
def transform_age(age):
    
    if age == np.nan:
        return 'unknown'
    
    elif age <= 18.0:
        return 'child'
    
    elif age <= 25:
        return 'adolescent'
    
    
    return 'adult'
    
X['Age'] = train['Age'].apply(transform_age)

In [16]:
expanses_col      = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
X['Expanses'] = train[expanses_col].sum(axis=1)

#adjust expenses (if expanses = 0 and one of the services is 0 -> 0)

for expanse in expanses_col:
    X.loc[(X[expanse].isna()) & (X['Expanses'] == 0), expanse] = 0

In [5]:
# if people are in CryoSleep they do not spend money => assume inverse holds as well. Makes sense because
# otherwise the persons would not even spend money for food
X.loc[(X['CryoSleep'].isna()) & (X['Expanses'] == 0) & (X['Age'] != 'child')] = True
X.loc[(X['CryoSleep'].isna()) & (X['Expanses'] != 0)] = False

In [ ]:
train_adapted.loc[(train_adapted['CryoSleep'].isna()) & (train_adapted['Expanses'] == 0),'CryoSleep'] = True
train_adapted.loc[(train_adapted['CryoSleep'].isna()) & (train_adapted['Expanses'] != 0),'CryoSleep'] = False

In [38]:
NB_Group = X.Group.value_counts().copy()
X['GroupSize'] = X.Group.replace(NB_Group.index, NB_Group)

1    4805
2     841
3     340
4     103
5      53
7      33
6      29
8      13
Name: Group, dtype: int64

In [46]:
group_missing_name = X.loc[(X['Name'] == 'Unknown') & (X['GroupSize'] >= 2)]['Group']

for group_nb in group_missing_name:
    
    group_names = X.loc[X['Group'] == group_nb, 'Name'].value_counts()
    
    # Count different names in group, -1 for the name 'Unknown' which is in all of them
    names_count = (group_names.shape)[0] - 1
    if names_count == 1:
        
        unique_name = (group_names.loc[group_names.index != 'Unknown'].index)[0]
        X.loc[X['Group'] == group_nb, 'Name'] = unique_name 
    
    else:
        most_frequent_names = group_names.loc[group_names == group_names.max()].index
        if len(most_frequent_names) == 1:
            X.loc[(X['Group'] == group_nb) & (X['Name'] == 'Unknown'), 'Name'] = most_frequent_names[0]

group_missing_name = X.loc[(X['Name'] == 'Unknown') & (X['GroupSize'] >= 2)]['Group']        

In [52]:
# These I do by hand
X[(X['Group'] == '2272') & (X['Name'] == 'Unknown'), 'Name'] = 'Kirks'
X[(X['Group'] == '2272') & (X['Name'] == 'Unknown'), 'Name'] = 'Horthy'